In [1]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-04-30 14:54:44--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-04-30 14:54:45--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-04-30 14:54:45 (77.8 MB/s) - written to stdout [1191/1191]

Installing

In [2]:
import os
import sys

import sparknlp

from sparknlp.base import *
from sparknlp.common import *
from sparknlp.annotator import *

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import pandas as pd

In [3]:
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv

--2023-04-30 13:03:59--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24032125 (23M) [text/plain]
Saving to: ‘news_category_train.csv.1’

news_category_train 100%[===================>]  22.92M  --.-KB/s    in 0.07s   

2023-04-30 13:03:59 (314 MB/s) - ‘news_category_train.csv.1’ saved [24032125/24032125]

--2023-04-30 13:03:59--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubus

In [3]:
import sparknlp
spark = sparknlp.start(gpu=True)

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 4.4.1
Apache Spark version: 3.2.3


In [5]:
raw_train_df = spark.read.option("header", True).csv("news_category_train.csv")
raw_test_df = spark.read.option("header", True).csv("news_category_test.csv")

In [6]:
raw_train_df.show(5)

+--------+--------------------+
|category|         description|
+--------+--------------------+
|Business| Short sellers, W...|
|Business| Private investme...|
|Business| Soaring crude pr...|
|Business| Authorities have...|
|Business| Tearaway world o...|
+--------+--------------------+
only showing top 5 rows



In [7]:

raw_train_df.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|   World|30000|
|Sci/Tech|30000|
|  Sports|30000|
|Business|30000|
+--------+-----+



In [8]:

raw_test_df.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
|category|count|
+--------+-----+
|   World| 1900|
|Sci/Tech| 1900|
|  Sports| 1900|
|Business| 1900|
+--------+-----+



# Bert w/o Preprocessing

In [10]:
! rm -r /root/annotator_logs

In [56]:
document_assembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
    .setInputCols(["document", "token"])\
    .setOutputCol("embeddings")

sentence_embedding = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierDL = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(5)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)\
    .setVerbose(1)


bert_no_processing_pipeline = Pipeline(stages=[document_assembler,
                                               tokenizer,
                                               bert_embeddings,
                                               sentence_embedding,
                                               classsifierDL])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [57]:
%time bert_no_processing_pipeline_model = bert_no_processing_pipeline.fit(raw_train_df)

CPU times: user 2 s, sys: 209 ms, total: 2.21 s
Wall time: 7min 39s


In [ ]:
os.listdir("/root/annotator_logs")

In [58]:
bert_no_processing_pipeline_model.write().overwrite().save('bert_no_processing_tokenized_pipeline_model')

In [ ]:
!zip -r bert_no_processing_tokenized_pipeline_model.zip bert_no_processing_tokenized_pipeline_model/

In [59]:
from sklearn.metrics import classification_report, accuracy_score

preds = bert_no_processing_pipeline_model.transform(raw_test_df)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))
print(accuracy_score(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

    Business       0.82      0.82      0.82      1900
    Sci/Tech       0.83      0.84      0.84      1900
      Sports       0.93      0.95      0.94      1900
       World       0.90      0.86      0.88      1900

    accuracy                           0.87      7600
   macro avg       0.87      0.87      0.87      7600
weighted avg       0.87      0.87      0.87      7600

0.8685526315789474


# Bert w/ Preprocessing

## Bert w/ Stopword Preprocessing

In [50]:
document_assembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
  
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols(["normalized"])\
    .setOutputCol("clean_token")\
    .setCaseSensitive(False)

bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
    .setInputCols(["document", "clean_token"])\
    .setOutputCol("embeddings")

sentence_embedding = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierDL = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(5)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)\
    .setVerbose(1)


bert_stopwords_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     normalizer,
                                     stopwords_cleaner,
                                     bert_embeddings,
                                     sentence_embedding,
                                     classsifierDL])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [51]:
%time bert_stopwords_pipeline_model = bert_stopwords_pipeline.fit(raw_train_df)

CPU times: user 1.94 s, sys: 241 ms, total: 2.19 s
Wall time: 7min 37s


In [53]:
bert_stopwords_pipeline_model.write().overwrite().save('bert_stopwords_pipeline_model')

In [ ]:
!zip -r bert_stopwords_pipeline_model.zip bert_stopwords_pipeline_model/

In [55]:
from sklearn.metrics import classification_report, accuracy_score

preds = bert_stopwords_pipeline_model.transform(raw_test_df)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))
print(accuracy_score(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

    Business       0.82      0.81      0.82      1900
    Sci/Tech       0.83      0.86      0.84      1900
      Sports       0.94      0.95      0.95      1900
       World       0.89      0.86      0.88      1900

    accuracy                           0.87      7600
   macro avg       0.87      0.87      0.87      7600
weighted avg       0.87      0.87      0.87      7600

0.87


## Bert w/ Lemmatized Preprocessing

In [61]:
document_assembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
  
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["normalized"]) \
      .setOutputCol("lemma")

bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
    .setInputCols(["document", "lemma"])\
    .setOutputCol("embeddings")

sentence_embedding = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierDL = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(5)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)\
    .setVerbose(1)


bert_lemma_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     normalizer,
                                     lemma,
                                     bert_embeddings,
                                     sentence_embedding,
                                     classsifierDL])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [62]:
%time bert_lemma_pipeline_model = bert_lemma_pipeline.fit(raw_train_df)

CPU times: user 2.32 s, sys: 282 ms, total: 2.6 s
Wall time: 9min 14s


In [63]:
bert_lemma_pipeline_model.write().overwrite().save('bert_lemma_pipeline_model')

In [ ]:
!zip -r bert_lemma_pipeline_model.zip bert_lemma_pipeline_model/

In [65]:
from sklearn.metrics import classification_report, accuracy_score

preds = bert_lemma_pipeline_model.transform(raw_test_df)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))
print(accuracy_score(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

    Business       0.82      0.80      0.81      1900
    Sci/Tech       0.81      0.85      0.83      1900
      Sports       0.93      0.95      0.94      1900
       World       0.90      0.85      0.87      1900

    accuracy                           0.86      7600
   macro avg       0.86      0.86      0.86      7600
weighted avg       0.86      0.86      0.86      7600

0.8639473684210527


## Bert w/ Stopwords and Lemma Preprocessing

In [66]:
document_assembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
  
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols(["normalized"])\
    .setOutputCol("clean_token")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["clean_token"]) \
      .setOutputCol("lemma")

bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
    .setInputCols(["document", "lemma"])\
    .setOutputCol("embeddings")

sentence_embedding = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierDL = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(5)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)\
    .setVerbose(1)


bert_stopwords_lemma_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     normalizer,
                                     stopwords_cleaner,
                                     lemma,
                                     bert_embeddings,
                                     sentence_embedding,
                                     classsifierDL])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[OK!]


In [67]:
%time bert_stopwords_lemma_pipeline_model = bert_stopwords_lemma_pipeline.fit(raw_train_df)

CPU times: user 2.19 s, sys: 285 ms, total: 2.48 s
Wall time: 8min 9s


In [68]:
bert_stopwords_lemma_pipeline_model.write().overwrite().save('bert_stopwords_lemma_pipeline_model')

In [ ]:
!zip -r bert_stopwords_lemma_pipeline_model.zip bert_stopwords_lemma_pipeline_model/

In [70]:
from sklearn.metrics import classification_report, accuracy_score

preds = bert_stopwords_lemma_pipeline_model.transform(raw_test_df)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))
print(accuracy_score(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

    Business       0.81      0.82      0.81      1900
    Sci/Tech       0.82      0.85      0.84      1900
      Sports       0.95      0.94      0.95      1900
       World       0.89      0.86      0.88      1900

    accuracy                           0.87      7600
   macro avg       0.87      0.87      0.87      7600
weighted avg       0.87      0.87      0.87      7600

0.8681578947368421


# Roberta

In [9]:
document_assembler = DocumentAssembler()\
    .setInputCol("description")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")
  
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
    .setInputCols(["normalized"])\
    .setOutputCol("clean_token")\
    .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["clean_token"]) \
      .setOutputCol("lemma")

roberta_embeddings = RoBertaEmbeddings.pretrained("roberta_embeddings_distilroberta_base",lang="en") \
    .setInputCols(["document", "lemma"]) \
    .setOutputCol("embeddings")

sentence_embedding = SentenceEmbeddings() \
    .setInputCols(["document", "embeddings"]) \
    .setOutputCol("sentence_embeddings") \
    .setPoolingStrategy("AVERAGE")

classsifierDL = ClassifierDLApproach()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("class")\
    .setLabelColumn("category")\
    .setMaxEpochs(5)\
    .setLr(0.001)\
    .setEnableOutputLogs(True)\
    .setVerbose(1)


roberta_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     normalizer,
                                     stopwords_cleaner,
                                     lemma,
                                     roberta_embeddings,
                                     sentence_embedding,
                                     classsifierDL])

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
roberta_embeddings_distilroberta_base download started this may take some time.
Approximate size to download 294.5 MB
[OK!]


In [10]:
%time roberta_pipeline_model = roberta_pipeline.fit(raw_train_df)

CPU times: user 8.14 s, sys: 949 ms, total: 9.09 s
Wall time: 31min 3s


In [11]:
roberta_pipeline_model.write().overwrite().save('roberta_pipeline_model')

In [12]:
!zip -r roberta_pipeline_model.zip roberta_pipeline_model/

  adding: roberta_pipeline_model/ (stored 0%)
  adding: roberta_pipeline_model/metadata/ (stored 0%)
  adding: roberta_pipeline_model/metadata/part-00000 (deflated 26%)
  adding: roberta_pipeline_model/metadata/_SUCCESS (stored 0%)
  adding: roberta_pipeline_model/metadata/._SUCCESS.crc (stored 0%)
  adding: roberta_pipeline_model/metadata/.part-00000.crc (stored 0%)
  adding: roberta_pipeline_model/stages/ (stored 0%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8d/ (stored 0%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8d/metadata/ (stored 0%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8d/metadata/part-00000 (deflated 40%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8d/metadata/_SUCCESS (stored 0%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8d/metadata/._SUCCESS.crc (stored 0%)
  adding: roberta_pipeline_model/stages/5_ROBERTA_EMBEDDINGS_7db024986e8

In [13]:
from sklearn.metrics import classification_report, accuracy_score

preds = roberta_pipeline_model.transform(raw_test_df)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))
print(accuracy_score(preds_df['category'], preds_df['result']))

              precision    recall  f1-score   support

    Business       0.83      0.83      0.83      1900
    Sci/Tech       0.84      0.86      0.85      1900
      Sports       0.93      0.97      0.95      1900
       World       0.92      0.85      0.88      1900

    accuracy                           0.88      7600
   macro avg       0.88      0.88      0.88      7600
weighted avg       0.88      0.88      0.88      7600

0.8777631578947368
